In [6]:
import pytesseract as pt
from PIL import Image
import platform

# from gtts import gTTS

import torch
import azure.cognitiveservices.speech as speechsdk
import json


In [7]:
if platform.system() == 'Darwin':  # macOS
    pt.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.4/bin/tesseract'
elif platform.system() == 'Windows':
    # Add your Windows-specific code here
    pass
else:
    print('Operating system not supported')

In [11]:
# get text from image
image = Image.open('text_images/pf-km-4.jpg')
text = pt.image_to_string(image)
print(text)

Anoriel Eight Eyes: "Alright, alright. This here is Scatvi, the daughter of my late friend Voltar... She's a cleric
of Abadar, just like her father. The man with the bow is Ellert. He's a silent type. The Lash is a bard. He can
whip you to tears with his words, hence the nickname. Finally, the Crimson Fury specializes in decapitating
anyone who isn't quick enough to hide from her. All four stand ready to dispatch a few dozen fey. Just say the
word."

1. "need loyal and experienced fighters."
2. "I'd like to use the tutor services of the Pathfinder Society."
3. "How did you end up here?"
+ 4. "I need your people's help.Scatvi, Ellert, Lash, Crimson Fury — I'd like to welcome you to my party."
5. "I have to go. Thank you, Anoriel."




In [12]:
text = text.replace('\n', ' ')
print(text)

Anoriel Eight Eyes: "Alright, alright. This here is Scatvi, the daughter of my late friend Voltar... She's a cleric of Abadar, just like her father. The man with the bow is Ellert. He's a silent type. The Lash is a bard. He can whip you to tears with his words, hence the nickname. Finally, the Crimson Fury specializes in decapitating anyone who isn't quick enough to hide from her. All four stand ready to dispatch a few dozen fey. Just say the word."  1. "need loyal and experienced fighters." 2. "I'd like to use the tutor services of the Pathfinder Society." 3. "How did you end up here?" + 4. "I need your people's help.Scatvi, Ellert, Lash, Crimson Fury — I'd like to welcome you to my party." 5. "I have to go. Thank you, Anoriel."  


In [19]:
# convert text to speech - Azure AI Speech

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = "7fe46879808446f092f8ad2eac44b746"
service_region = "uksouth"

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
# Note: the voice setting will not overwrite the voice element in input SSML.
speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"

audio_config = speechsdk.audio.AudioOutputConfig(filename="audio_output/ai-pf-km-4.wav")

# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

result = speech_synthesizer.speak_text_async(text).get()
# Check result
if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))



Speech synthesized for text [Anoriel Eight Eyes: "Alright, alright. This here is Scatvi, the daughter of my late friend Voltar... She's a cleric of Abadar, just like her father. The man with the bow is Ellert. He's a silent type. The Lash is a bard. He can whip you to tears with his words, hence the nickname. Finally, the Crimson Fury specializes in decapitating anyone who isn't quick enough to hide from her. All four stand ready to dispatch a few dozen fey. Just say the word."  1. "need loyal and experienced fighters." 2. "I'd like to use the tutor services of the Pathfinder Society." 3. "How did you end up here?" + 4. "I need your people's help.Scatvi, Ellert, Lash, Crimson Fury — I'd like to welcome you to my party." 5. "I have to go. Thank you, Anoriel."  ]


In [ ]:
# convert text to speech - gTTS
# tts = gTTS(text, lang='en')
# tts.save('audio_output/gtts-pf-km-4.mp3')

NameError: name 'gTTS' is not defined

In [30]:
# API version

import requests
import json

subscription_key = "7fe46879808446f092f8ad2eac44b746"
service_region = "uksouth"
tts_service_url = f'https://{service_region}.tts.speech.microsoft.com/cognitiveservices/v1'

fetch_token_url = f'https://{service_region}.api.cognitive.microsoft.com/sts/v1.0/issueToken'

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-Type': 'application/ssml+xml',
    'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm'
}

token_response = requests.post(fetch_token_url, headers={'Ocp-Apim-Subscription-Key': subscription_key})
access_token = str(token_response.text)

# Update the headers to use the token
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/ssml+xml',
    'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm'
}

ssml_text = f"""
<speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-US'>
    <voice name='en-US-AriaNeural'>Hello friend. We are going on an adventure. Buckle up!</voice>
</speak>
"""


response = requests.post(tts_service_url, headers=headers, data=ssml_text)

if response.status_code == 200:
    with open('audio_output/output_audio.wav', 'wb') as audio_file:
        audio_file.write(response.content)
    print("Audio saved to 'output_audio.wav'.")
else:
    print(f"Error: {response.status_code} - {response.reason}")


Audio saved to 'output_audio.wav'.
